# Marriage Perception
## Exploratory Analysis

In [ ]:
# Import relevant Python libraries here
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [ ]:
# Load dataset into notebook
df = pd.read_csv('Marriage Perception (Responses) - Form Responses 1.csv')

# Shape of dataset
print(f"Size of Dataset (Rows, Cols): {df.shape}\n")

# Peek at head of dataset
print("Preview: ")
df.head()

In [ ]:
# Check info
print(df.info())

In [ ]:
# Check for duplicates
print(f"Duplicates: {df.duplicated().sum()}\n")

# Check for null values
total_missing = df.isna().sum().sum()
col_missing = df.isna().sum(axis=0)
row_missing = df.isna().sum(axis=1)

print(f"Total missing values: {total_missing}\n")
print(f"Missing per row:\n{row_missing}\n")
print(f"Missing per column:\n{col_missing}")


## Manipulating and Cleansing Initial Dataset

In [ ]:
## Function to extract numerical digits from Likert Scale strings
def clean_ratings(val):
    if pd.isna(val):
        return val

    # Use regex to find first digit in string
    match = re.search(r'(\d)', str(val))
    if match:
        return int(match.group(1))
    return val

# Indentify columns to clean
col_review = df.columns[1:22]
for col in col_review:
    df[col] = df[col].apply(clean_ratings)

## Shorten column names for simplicity
column_mapping = {
    df.columns[1]: 'want_married',
    df.columns[2]: 'unique_benefits',
    df.columns[3]: 'everyone_married',
    df.columns[4]: 'more_than_legal',
    df.columns[5]: 'outdated_idea',
    df.columns[6]: 'happy_unmarried',
    df.columns[7]: 'goal_is_marriage',
    df.columns[8]: 'unique_challenges',
    df.columns[9]: 'highest_level_rel',
    df.columns[10]: 'do_not_want_marry',
    df.columns[11]: 'marriage_important',
    df.columns[12]: 'money_success_ind',
    df.columns[13]: 'money_measure_success',
    df.columns[14]: 'rich_achievement',
    df.columns[15]: 'anxious_finances',
    df.columns[16]: 'good_budgeting',
    df.columns[17]: 'hesitant_change',
    df.columns[18]: 'uncertainty_nerve_wracking',
    df.columns[19]: 'variety_spice_life',
    df.columns[20]: 'map_out_career',
    df.columns[21]: 'taking_chances',
    df.columns[22]: 'age',
    df.columns[23]: 'gender',
    df.columns[24]: 'orientation',
    df.columns[25]: 'is_married',
    df.columns[26]: 'religion_affiliation',
    df.columns[27]: 'religiosity',
}

df = df.rename(columns=column_mapping)

## Remove empty questions column
if df.columns[-1].startswith('If you have any questions or comments'):
    df = df.drop(columns=[df.columns[-1]])


## Create aggregate score for valuation of money (OPTIONAL?)
df['money_value_score'] = df[['money_success_ind', 'money_measure_success', 'rich_achievement']].mean(axis=1) # combines questions pertaining to money

### Removing Biased Entries
Some mobile users possibly only noticed 3-4 of the 7 Likert Scale options available, hence choosing only up to 3-4.

In [ ]:
## Identify all columns
likert_columns = [
    'want_married', 'unique_benefits', 'everyone_married', 'more_than_legal', 
    'outdated_idea', 'happy_unmarried', 'goal_is_marriage', 'unique_challenges', 
    'highest_level_rel', 'do_not_want_marry', 'marriage_important', 'money_success_ind', 
    'money_measure_success', 'rich_achievement', 'anxious_finances', 'good_budgeting', 
    'hesitant_change', 'uncertainty_nerve_wracking', 'variety_spice_life', 
    'map_out_career', 'taking_chances'
]

## Calculate max rating given by each respondent
df['row_max'] = df[likert_columns].max(axis=1)

## Flag any potential mobile bias
# Highly Suspicious = max rating of 3
# Suspicious = max rating of 4
df['bias_flag'] = 'None'
df.loc[df['row_max'] <= 4, 'bias_flag'] = 'Suspicious (Max 4)'
df.loc[df['row_max'] <= 3, 'bias_flag'] = 'Highly Suspicious (Max 3)'

## Isolate into separate dataframe
df_suspects = df[df['bias_flag'] != 'None']

## Print statements
print(f"Total entries: {len(df)}")
print(f"Suspect entries identified: {len(df_suspects)}")
df_suspects.head()

In [ ]:
## Display new, clean dataset
print("Data cleaning complete.\n")
print(f"New shape: {df.shape}")
df.head(10)

In [ ]:
# Filter out biased entries
df_clean = df[df['bias_flag'] == 'None'].copy()

print(f"Original entries: {len(df)}")
print(f"Clean entries for visualization: {len(df_clean)}")

## Visualizing Data

In [ ]:
# Selecting the most important 
marriage_cols = ['want_married', 'unique_benefits', 'everyone_married', 'outdated_idea', 'marriage_important', 'money_value_score', 'religiosity']

plt.figure(figsize=(10, 8))
corr = df_clean[marriage_cols].corr()

# Create a mask to hide the top triangle (make it easier to read)
mask = np.triu(np.ones_like(corr, dtype=bool))

# Plot heatmap
sns.heatmap(corr, mask=mask, annot=True, cmap='RdBu_r', center=0, fmt=".2f")
plt.title('Correlation Between Marriage, Money, and Religiosity')
plt.show()